# JESS Web Users Functions API Example
This notebook provides a demonstration of the user's functions of the JESS Web API. For documentation of the API, please visit: [JESS Web API](http://www.jeplus.org/wiki/doku.php?id=docs:jea:jea_api "Go to JEA API docs"). 

Let's start with loading the necessary package and testing the connection to the server. The "[Requests: HTTP for Humans](http://python-requests.org "Go to Python Requests website")" is shown here, although other http packaages can be used too. 

After loading the Request package, we can run the Info command. Info is the simplest command and does not require authentication, so it works also if you copy and paste the URL to a browser, or [click here](https://api.ensims.com/users/api/info "Try Info command in browser").

In [1]:
# Use requests (see https://requests.readthedocs.io/en/master/)
import requests

# API endpoint URL
ApiBase = 'https://api.ensims.com/'
# ApiBase = 'https://localhost:8443/'
UserApi = ApiBase + 'users/api/'

# Test connection
r = requests.get(UserApi + 'info')
r.json()

{'Title': 'Users Web API',
 'Description': 'JESS User Management API provided by ENSIMS Ltd.',
 'Major': 1,
 'Minor': 0,
 'Revision': 0,
 'Release': 'beta',
 'Update': 0,
 'Notice': '(C) 2017, Energy Simulation Solutions Ltd. All rights reserved.'}

Requests will make a GET request to the given URL, and store the returned data in 'r'. Most of the data returned by the users API are in JSON format, as shown above. You can access individual field like this:

In [2]:
r.json()['Title']

'Users Web API'

## Authentication

In order to do anything useful, you must first create an account and log on to the system by acquiring a session token with your credentials. You can create a JESS Web account on [app.ensims.com](https://app.ensims.com/ "Go to app.ensims.com"). It is possible to create an account using the API functions only. Please refer to the documentation for details.

Once you have got an account, use the auth command to log on to the system. The auth command is a POST request with appropriate headers and the user credentials in JSON format as the data body. Here is how you do it. **Please remember to replace the login email with your own.** Also, since your password will be sent in clear text, please make sure that you do NOT show it in any publically accessible scripts, and use only the HTTPS protocol when accessing the JESS Web platform.

In this example, we use another library for typing in the password.

In [17]:
# Get log in credential
import getpass
user_email = "yi@jeplus.org"
password = getpass.getpass("Password for " + user_email + ": ")

# Set header and body of the POST request
headers = {'Content-Type': 'application/json'}
body = {"email": user_email, "password": password}

# Send request
r = requests.post(UserApi + 'auth', headers=headers, json=body)

# Keep the cookies
cookies = r.cookies

# Check data returned by JEA
r.json()

Password for yi@jeplus.org:  ········


{'ok': True,
 'status': 'Logged in successfully!',
 'jwt': 'Session token in cookie',
 'user': 'Yi',
 'role': 'pro',
 'email': 'yi@jeplus.org'}

The JESS Web platform uses JSON Web Tokens (JWT) as the authentication mechanism. If logged on successfully, a JWT key used as the session token will be returned by the service as a cookie. The session token needs to be kept safe and sent with any commands requiring authentication. Here is how you get the session token cookie and view its contents. Please note that the token has a limited life span of a few hours, during which time it must be kept a secrete.

To check if a session is still valid, we can use the 'checkin' command. This command also demonstrates how the session token cookie being used for authentication in the commands that require it. If 'checkin' with the existing token is successful, a new token will be returned hence we can extend the life span of the current session. Just remember to store the updated cookies.

In [7]:
# Check-in
r = requests.post(UserApi + 'checkin', cookies=cookies)

# Update the copy of the key if successful
if r.json()['ok']:
    cookies = r.cookies

# Show the response
r.json()

{'ok': True,
 'status': 'Session renewed successfully!',
 'jwt': 'Session key cookie',
 'user': 'Yi',
 'role': 'user',
 'email': 'yi@jeplus.org'}

## Upload, download, and view files

We can upload files to a storage area linked to this user account. These files can be used for simulation (see JESS API) or optimisation projects (see JEA API) later. With certain file types, e.g. the EPW, ESO and IDF files, we can also use this facility as a viewer. 

(**Please note** that the upload area for ENSIMS Web Service users are not backed up, hence it should be used as a transient storage. ENSIMS is not responsible for any file losses.)


To upload a file to a specific folder, we can use POST files/

In [18]:
# upload a file to a particular folder
folder = "test/0"
files = {'file': open('c:\\windows\\win.ini', 'rb')}

# POST with files
r = requests.post(UserApi + 'files/' + folder, files=files, cookies=cookies)

# Show the returned status
r.json()

{'ok': True,
 'status': 'File(s) uploaded successfully',
 'fileHandle': 'test/0',
 'callback': '/users/api/files/test/0'}

We can view the contents of the upload area, or a sub-folder in there, using the `GET files/` comamnd. It will retrieve the contents of the specified folder in a tree structure:

In [19]:
# retrieve files in a particular folder
folder = "test/0"
r = requests.get(UserApi + 'files/' + folder, cookies=cookies)

# Show the returned status
r.json()

{'fileName': '/test/0',
 'open': True,
 'nocheck': True,
 'fileSize': 0,
 'children': [{'fileName': 'win.ini',
   'open': False,
   'nocheck': False,
   'fileSize': 167,
   'children': [],
   'name': 'win.ini (0.17 KB)'}],
 'name': '/test/0'}

The same command downloads the file if the paths are pointing to a file instead of a folder:

In [20]:
# retrieve a file in a particular folder
filepath = "test/0/win.ini"
r = requests.get(UserApi + 'files/' + filepath, cookies=cookies)

# Show the contents of the downloaded file
print(r.text)

; for 16-bit app support
[fonts]
[extensions]
[mci extensions]
[files]
[Mail]
MAPI=1
CMCDLLNAME32=mapi32.dll
CMC=1
MAPIX=1
MAPIXVER=1.0.0.1
OLEMessaging=1



When a zip archive containing multiple files and folders is uploaded, its contents will be extracted automatically and stored in the user's upload area, as shown in the example here. Please note the returned file handle may be the target folder in the user's area, or the root folder of the zip file residing in the target folder, depending on how the zip archive is created.

In [23]:
# upload a file to a particular folder
folder = ""
files = {'file': open('job_example\\Shoebox_v8.9.zip', 'rb')}

# POST with files
r = requests.post(UserApi + 'files/' + folder, files=files, cookies=cookies)

# Show the returned status
r.json()

{'ok': True,
 'status': 'File(s) uploaded successfully',
 'fileHandle': 'Shoebox_v8.9',
 'callback': '/users/api/files/Shoebox_v8.9'}

The contents of the zip archive will be extracted to `Shoebox_v89`. We can check its contents.

In [24]:
# retrieve files in a particular folder
folder = r.json()['fileHandle']
r = requests.get(UserApi + 'files/' + folder, cookies=cookies)

# Show the returned status
r.json()

{'fileName': '/Shoebox_v8.9',
 'open': True,
 'nocheck': True,
 'fileSize': 0,
 'children': [{'fileName': '2030_Heathrow_a1fi_50_percentile_TRY.epw',
   'open': False,
   'nocheck': False,
   'fileSize': 1439809,
   'children': [],
   'name': '2030_Heathrow_a1fi_50_percentile_TRY.epw (1.44 MB)'},
  {'fileName': '2050_Heathrow_a1fi_50_percentile_TRY.epw',
   'open': False,
   'nocheck': False,
   'fileSize': 1440039,
   'children': [],
   'name': '2050_Heathrow_a1fi_50_percentile_TRY.epw (1.44 MB)'},
  {'fileName': '2080_Heathrow_a1fi_50_percentile_TRY.epw',
   'open': False,
   'nocheck': False,
   'fileSize': 1441330,
   'children': [],
   'name': '2080_Heathrow_a1fi_50_percentile_TRY.epw (1.44 MB)'},
  {'fileName': 'cntr_Heathrow_TRY.epw',
   'open': False,
   'nocheck': False,
   'fileSize': 1438895,
   'children': [],
   'name': 'cntr_Heathrow_TRY.epw (1.44 MB)'},
  {'fileName': 'joblist_out.csv',
   'open': False,
   'nocheck': False,
   'fileSize': 4837,
   'children': [],
   'na

To delet a file or a folder with everything in it, use the DELETE files/ command:

In [25]:
# delete a file from the given folder
filepath = 'Shoebox_v8.9'

# DELETE with files
r = requests.delete(UserApi + 'files/' + filepath, cookies=cookies)

# Show the returned status
r.json()

{'ok': True, 'status': 'Shoebox_v8.9 has been deleted'}

Finally, ENSIMS Web Service currently provides viewers for EPW, ESO, IDF and DXF files. Their uses will be shown in separate demos. 

## End of Demo
